# **Prediction of Product Sales**




### *Author: Matt S.*

## Project Overview

# **Issues**
- Clean data #7
  - df.duplicates().sum() returns zero
  - df['Item_Identifier'].duplicates.sum() returns over 6000 values
    - When using nunique(), info(), and checking the Item Identfier column specifically, the math adds up to the amount of rows for the dataframe.

How should this be handled? Identifier is supposed to be unique, but is only 18% unique. Additionally, values cannot be filled with a mean because the column consists of strings.

## Load and Inspect Data

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import required packages
import pandas as pd

# Save the filepath and load the data as "df_titanic"
file_url = "/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv"

df = pd.read_csv(file_url)

### Preview the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Clean Data

### 1) How many rows and columns?

In [5]:
df.shape

(8523, 12)

### 2) What are the datatypes of each variable?



In [6]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

### 3) Are there duplicates? If so, drop any duplicates.



In [7]:
duplicated_rows = df.duplicated()
duplicated_rows.sum()

0

### 4) Identify missing values.



In [8]:
cols_with_null = df.isna()
cols_with_null.sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### 5) Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).



1. Weight will be dropped be dropped.
  - It is worth considering dropping because there is a large amount of missing rows. However, it is 1463 rows out of 8523 rows. Overall, not that many rows are missing. The rows might be able to be filled in.

2. Outlet size may be kept or drop. The available columns are a mix of individual product weight and building details.
  - Outlet size needs to be cleaned at least, it is currently showing up as an object.

For now, all missing values will be updated to MISSING.

In [53]:
# Filter for number columns
num_cols = df.select_dtypes('number').columns

In [9]:
# Set null values in Item_Weight to -1
df[num_cols] = df[num_cols].fillna(-1)

In [10]:
# Filter for null weight values
cat_cols = df.select_dtypes('object').columns

# null_weight_values = df['Item_Weight'].isna()

# Set null values in Item_Weight to -1
df[cat_cols] = df[cat_cols].fillna("MISSING")

### 6) Confirm that there are no missing values after addressing them.



In [11]:
# Check updated null weight values
num_cols.isna().sum()

0

In [12]:
# Check updated null weight values
cat_cols.isna().sum()

0

In [13]:
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [55]:
# Check for unique category values
df[cat_cols].nunique()

Item_Identifier         1559
Item_Fat_Content           5
Item_Type                 16
Outlet_Identifier         10
Outlet_Size                4
Outlet_Location_Type       3
Outlet_Type                4
dtype: int64

In [56]:
# Check for unique numerical values
df[num_cols].nunique()

Item_Weight                   416
Outlet_Establishment_Year       9
Item_Outlet_Sales            3493
dtype: int64

In [57]:
# Use the result to determine if any of the columns have 100% unique values.
df.nunique()/len(df) * 100

Item_Identifier              18.291681
Item_Weight                   4.880910
Item_Fat_Content              0.058665
Item_Type                     0.187727
Outlet_Identifier             0.117330
Outlet_Establishment_Year     0.105597
Outlet_Size                   0.046932
Outlet_Location_Type          0.035199
Outlet_Type                   0.046932
Item_Outlet_Sales            40.983222
dtype: float64

- Item_Visibility should be dropped because its data is irrelevant and has many unique values while not being the index.

- Item_MRP will also be dropped, there isn't any real relation to the rest of the columns either.

In [17]:
# Drop the Item_Visiblity and Item_MRP columns
df.drop(columns = ['Item_Visibility', 'Item_MRP'], inplace = True)

In [21]:
# Confirm that Item_Visiblity and MRP are gone.
for col in df.columns: print(col)

Item_Identifier
Item_Weight
Item_Fat_Content
Item_Type
Outlet_Identifier
Outlet_Establishment_Year
Outlet_Size
Outlet_Location_Type
Outlet_Type
Item_Outlet_Sales


### 7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent).



In [ ]:
# Obtain the value counts for all string columns
for col in cat_cols:
  print(f"Value Counts for {col}")
  print(df[col].value_counts())
  print("\n")



In [52]:
df[cat_cols].nunique()
df.nunique() / len(df) * 100
# duplicated_rows = df.duplicated()
# duplicated_rows.sum()

# Item_Identifier is a problematic column, it needs to be cleaned up. 
#It doesn't have duplicates and it doesn't have null values, yet it is only 18% unique.
df['Item_Identifier'].dtype
df['Item_Identifier'].duplicated().sum()

# 8523 - 6964 = 1559, which is the current count of unique vals.

# Not sure why df.duplicated.sum returns zero but focusing on a specific column returns a proper value.

6964

### 8) For any numerical columns, obtain the summary statistics of each (min, max, mean).


In [54]:
df[num_cols].describe()

,Item_Weight,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000
mean,10.478936,1997.831867,2181.288914
std,6.720742,8.371760,1706.499616
min,-1.000000,1985.000000,33.290000
25%,6.650000,1987.000000,834.247400
50%,11.000000,1999.000000,1794.331000
75%,16.000000,2004.000000,3101.296400
max,21.350000,2009.000000,13086.964800


## Exploratory Data Analysis

## Explanatory Data Analysis